In [56]:
using LinearAlgebra 
using Plots #Graph
using StaticArrays 
using SparseArrays  
using Optim
using Arpack  #Eigenvalues and eigenvectors
using LaTeXStrings #Titles and labels 
using JLD #Save data
#definition of N,M,D
global N=4
global M=4
global D=Int((factorial(M+N-1))/(factorial(M-1)*factorial(N)))  

#i-prime number function(suggested form) 
p(i)=100i+3 

#Generate a basis for N = M
function generate_basis(N, M)
    D = prod(max(N, M):N+M-1) ÷ prod(1:min(N, M))
    basis = [zeros(Int, M) for _ in 1:D]
    basis[1][1] = N
    for t = 2:D
        if basis[t-1][M] != 0
            k = M - 1
        else
            k = M
        end
        while k > 0 && basis[t-1][k] == 0
            k -= 1
        end
        @views basis[t][1:k-1] .= basis[t-1][1:k-1]
        basis[t][k] = basis[t-1][k] - 1
        basis[t][k+1] = N - sum(@view(basis[t][1:k])) 
        end 
    return basis  
end

generate_basis (generic function with 1 method)

In [57]:
v=generate_basis(N,M)

35-element Vector{Vector{Int64}}:
 [4, 0, 0, 0]
 [3, 1, 0, 0]
 [3, 0, 1, 0]
 [3, 0, 0, 1]
 [2, 2, 0, 0]
 [2, 1, 1, 0]
 [2, 1, 0, 1]
 [2, 0, 2, 0]
 [2, 0, 1, 1]
 [2, 0, 0, 2]
 [1, 3, 0, 0]
 [1, 2, 1, 0]
 [1, 2, 0, 1]
 ⋮
 [0, 2, 2, 0]
 [0, 2, 1, 1]
 [0, 2, 0, 2]
 [0, 1, 3, 0]
 [0, 1, 2, 1]
 [0, 1, 1, 2]
 [0, 1, 0, 3]
 [0, 0, 4, 0]
 [0, 0, 3, 1]
 [0, 0, 2, 2]
 [0, 0, 1, 3]
 [0, 0, 0, 4]

In [58]:
function occupation(i, v) 
     if(v[i] >= 1)  
        return  v[i]  
    else 
        return 0;
    end  
end 

occupation (generic function with 1 method)

In [59]:
function matrixoccupation(k, D) 
    mat = spzeros(Complex{Float64}, D,D)
    for j in 1:D 
        mat[j,j] = occupation(k, v[j]) 
    end   
    return mat 
end

matrixoccupation (generic function with 1 method)

In [60]:
#sum( matrixoccupation(i,D) for i in 2:(N-1) ) == matrixoccupation(2,D)

In [61]:
#Parity order parameter 
theta = pi
Op = exp(1im*theta*Matrix(sum(matrixoccupation(i,D) for i in 1:(N-1))) - (N-1)*I) 
Op = real(Op)

35×35 Matrix{Float64}:
 0.0497871  0.0        0.0        …  0.0         0.0        0.0
 0.0        0.0497871  0.0           0.0         0.0        0.0
 0.0        0.0        0.0497871     0.0         0.0        0.0
 0.0        0.0        0.0           0.0         0.0        0.0
 0.0        0.0        0.0           0.0         0.0        0.0
 0.0        0.0        0.0        …  0.0         0.0        0.0
 0.0        0.0        0.0           0.0         0.0        0.0
 0.0        0.0        0.0           0.0         0.0        0.0
 0.0        0.0        0.0           0.0         0.0        0.0
 0.0        0.0        0.0           0.0         0.0        0.0
 0.0        0.0        0.0        …  0.0         0.0        0.0
 0.0        0.0        0.0           0.0         0.0        0.0
 0.0        0.0        0.0           0.0         0.0        0.0
 ⋮                                ⋱                         
 0.0        0.0        0.0           0.0         0.0        0.0
 0.0        0.0     

In [62]:
#String Order parameter (we fix the parameters i =1, j = N, and i<=k<j )
Os = (Matrix(matrixoccupation(1,D)) - I)*(exp(1im*theta*Matrix(sum(matrixoccupation(i,D) for i in 1:(N-1))) - (N-1)*I))*(Matrix(matrixoccupation(N,D)) - I) 
Os = real(Os)

35×35 Matrix{Float64}:
 -0.149361   0.0         0.0        0.0  …   0.0        0.0         0.0
  0.0       -0.0995741   0.0        0.0      0.0        0.0         0.0
  0.0        0.0        -0.0995741  0.0      0.0        0.0         0.0
  0.0        0.0         0.0        0.0      0.0        0.0         0.0
  0.0        0.0         0.0        0.0      0.0        0.0         0.0
  0.0        0.0         0.0        0.0  …   0.0        0.0         0.0
  0.0        0.0         0.0        0.0      0.0        0.0         0.0
  0.0        0.0         0.0        0.0      0.0        0.0         0.0
  0.0        0.0         0.0        0.0      0.0        0.0         0.0
  0.0        0.0         0.0        0.0      0.0        0.0         0.0
  0.0        0.0         0.0        0.0  …   0.0        0.0         0.0
  0.0        0.0         0.0        0.0      0.0        0.0         0.0
  0.0        0.0         0.0        0.0      0.0        0.0         0.0
  ⋮                                      

In [63]:
#Optim Example for 
#f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2  
#x0 = [0.0, 0.0]  
#optimize(f, x0)  
# optimize(f, zeros(2) )  
##optimize(x->dot(x, Os*x), zeros(D)) ###NO CONSTRAINTS! so we will use JuMP package
using JuMP
import Ipopt

In [64]:

function ParityOrder(; verbose = true)
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, z)
    @variable(model, x[1:D] >= 0)
    @objective(model, Max, z) 
    @constraint(model, dot(x, Op*x) == z)
    @constraint(model, sum(x.^2) == 1)
    optimize!(model)
    if verbose
        print(model)
        println("Objective value: ", objective_value(model))
        println("Op = ", value(z))
        for i in 1:D 
            println("x_$i = ", value(x[i]) ) 
        end
    end
    return
end

ParityOrder()

Objective value: 0.049787052080982046
Op = 0.049787052080982046
x_1 = 0.21320069897334865
x_2 = 0.21320069897334865
x_3 = 0.21320069897334865
x_4 = 0.00011485435804889359
x_5 = 0.21320069897334865
x_6 = 0.21320069897334865
x_7 = 0.00011485435804889359
x_8 = 0.21320069897334865
x_9 = 0.00011485435804889359
x_10 = 0.21320069877583475
x_11 = 0.21320069897334865
x_12 = 0.21320069897334865
x_13 = 0.00011485435804889359
x_14 = 0.21320069897334865
x_15 = 0.00011485435804889359
x_16 = 0.21320069877583475
x_17 = 0.21320069897334865
x_18 = 0.00011485435804889359
x_19 = 0.21320069877583475
x_20 = 0.00011485435804889358
x_21 = 0.21320069897334865
x_22 = 0.21320069897334865
x_23 = 0.00011485435804889359
x_24 = 0.21320069897334865
x_25 = 0.00011485435804889359
x_26 = 0.21320069877583475
x_27 = 0.21320069897334865
x_28 = 0.00011485435804889359
x_29 = 0.21320069877583475
x_30 = 0.00011485435804889358
x_31 = 0.21320069897334865
x_32 = 0.00011485435804889359
x_33 = 0.21320069877583475
x_34 = 0.000114854

In [65]:
function StringOrder(; verbose = true)
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, z)
    @variable(model, x[1:D] >= 0)
    @objective(model, Max, z) 
    @constraint(model, dot(x, Op*x) == z)
    @constraint(model, sum(x.^2) == 1)
    optimize!(model)
    if verbose
        print(model)
        println("Objective value: ", objective_value(model))
        println("Os = ", value(z))
        for i in 1:D 
            println("x_$i = ", value(x[i]) ) 
        end
    end
    return
end

StringOrder()

Objective value: 0.049787052080982046
Os = 0.049787052080982046
x_1 = 0.21320069897334865
x_2 = 0.21320069897334865
x_3 = 0.21320069897334865
x_4 = 0.00011485435804889359
x_5 = 0.21320069897334865
x_6 = 0.21320069897334865
x_7 = 0.00011485435804889359
x_8 = 0.21320069897334865
x_9 = 0.00011485435804889359
x_10 = 0.21320069877583475
x_11 = 0.21320069897334865
x_12 = 0.21320069897334865
x_13 = 0.00011485435804889359
x_14 = 0.21320069897334865
x_15 = 0.00011485435804889359
x_16 = 0.21320069877583475
x_17 = 0.21320069897334865
x_18 = 0.00011485435804889359
x_19 = 0.21320069877583475
x_20 = 0.00011485435804889358
x_21 = 0.21320069897334865
x_22 = 0.21320069897334865
x_23 = 0.00011485435804889359
x_24 = 0.21320069897334865
x_25 = 0.00011485435804889359
x_26 = 0.21320069877583475
x_27 = 0.21320069897334865
x_28 = 0.00011485435804889359
x_29 = 0.21320069877583475
x_30 = 0.00011485435804889358
x_31 = 0.21320069897334865
x_32 = 0.00011485435804889359
x_33 = 0.21320069877583475
x_34 = 0.000114854